# Tweepy

In [ ]:
!pip install tweepy
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=853f1f05275de93b3e916c02c1b121bb0aa199ad7b3720a79e7c7cc0c37d6f23
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
#imports
import json
import tweepy as tw
from datetime import datetime, timedelta
import pandas as pd
import pytz
import pandas as pd
import re
from pyspark import SparkContext, SparkConf
import csv

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth,wait_on_rate_limit=True)

NameError: ignored

# Get Followers List

In [ ]:
#get user ID from screen name
screen_name = ""
user = api.get_user(screen_name = screen_name).id_str

#Get list of accounts the user is following
following_list = []
try:
  for page in tw.Cursor(api.get_friend_ids, user_id=user).pages():
    following_list.extend(page)
except tw.errors.TweepyException:
  print("error")
len(following_list)

145

# Fetch tweets from following list
## trying out - not part of pipeline

Fetch tweets from one account

In [ ]:
##how to get user id from a username
user_id = api.get_user(screen_name = screen_name).id_str
user_id

In [ ]:

user = api.get_user(user_id='39125788')
#user = api.get_user(user_id=following_list[0])
user.screen_name

'FABRICtestbed'

In [ ]:
##trying on an individual account - not part of pipeline

tz = pytz.timezone('US/Central')
now = datetime.now(tz)
yesterday = now - timedelta(days=1)
tweets=[]
all_tweets = []
tweet_dict = {}
user_id = following_list[1]

for tweet in tw.Cursor(api.user_timeline, user_id=user_id, exclude_replies=True,tweet_mode='extended').items():

    if tweet.created_at >= yesterday and tweet.lang=="en":
      # tweets.append(tweet.full_text)
      #print(tweet.full_text)
      tweet_id = tweet.id_str
      #name = tweet.user.name
      tweet_dict['user_id']=user_id
      tweet_dict['tweet_id']=tweet_id
      tweet_dict['full_text']=tweet.full_text

      replies_itterator = tw.Cursor(api.search_tweets, q='to:{}'.format(api.get_status(tweet_id).user.screen_name),
                          since_id=tweet_id, tweet_mode='extended').items()
      replies = []
      for reply in replies_itterator:
        if hasattr(reply, 'in_reply_to_status_id_str'):
          if reply.in_reply_to_status_id_str == tweet_id:
            replies.append(re.sub(r"@\w+\s?", "", reply.full_text))
            #print(reply.full_text)

      tweet_dict['replies']=replies

    else:
      break

    all_tweets.append(tweet_dict)

In [ ]:
len(all_tweets)

6

In [ ]:
all_tweets

In [ ]:
with open("tweets.json", "w") as f:
    json.dump(all_tweets, f)

In [ ]:
all_tweets[0]['replies'][0]

'@OSINT_Insider @TwitterBlue Upto 7 days!'

In [ ]:
for user in following_list[0:4]:
  user = api.get_user(user_id=user)
  print(user.screen_name)

MaryFloraHunter
emollick
FABRICtestbed
networkservice


# Fetch tweets recursively

In [ ]:
# test removing @ mentions

text = "@OSINT_Insider @TwitterBlue Upto 7 days!"
re.sub(r"@\w+\s?", "", text)

'Upto 7 days!'

In [ ]:
#recurrsively fetches tweets for the following accounts and their replies posted in last 24 hours

tz = pytz.timezone('US/Central')
now = datetime.now(tz)
yesterday = now - timedelta(days=1)
tweets=[]
all_tweets = []

##running on 1 account for demo
for user in following_list[1:2]:
  print("getting tweet for following user ",api.get_user(user_id=user).screen_name)
  for tweet in tw.Cursor(api.user_timeline, user_id=user, exclude_replies=True,tweet_mode='extended').items():
      tweet_dict = {}
      if tweet.created_at >= yesterday and tweet.lang=="en":
        # tweets.append(tweet.full_text)
        #print(tweet.full_text)
        tweet_id = tweet.id_str
        #name = tweet.user.name
        tweet_dict['user_id']=user_id
        tweet_dict['user_name']=tweet.user.screen_name
        tweet_dict['tweet_id']=tweet_id
        tweet_dict['full_text']=tweet.full_text

        replies_itterator = tw.Cursor(api.search_tweets, q='to:{}'.format(api.get_status(tweet_id).user.screen_name),
                            since_id=tweet_id, tweet_mode='extended').items()
        replies = []
        for reply in replies_itterator:
          if hasattr(reply, 'in_reply_to_status_id_str'):
            if reply.in_reply_to_status_id_str == tweet_id:
              if reply.lang == "en":
                replies.append(re.sub(r"@\w+\s?", "", reply.full_text))
              #print(reply.full_text)

        tweet_dict['replies']=replies
        all_tweets.append(tweet_dict)
      else:
        break

getting tweet for following user  yoursunny


In [ ]:
all_tweets

[{'user_id': 22088130,
  'user_name': 'yoursunny',
  'tweet_id': '1655907894603268099',
  'full_text': 'Car returned. Total distance is 1213 miles. (@ Nextcar Gaithersburg South in Gaithersburg, MD) https://t.co/3hBzg8pvFq'},
 {'user_id': 22088130,
  'user_name': 'yoursunny',
  'tweet_id': '1655896726744494082',
  'full_text': 'Look for evidence of a dry-land plane crash in 1957. (@ Little Bennett Regional Park - @montgomeryparks in Clarksburg, MD) https://t.co/3iDolTAlVy https://t.co/EHmvlOjw5P'},
 {'user_id': 22088130,
  'user_name': 'yoursunny',
  'tweet_id': '1655695863878959105',
  'full_text': 'Open! (@ Cronise Market Place in Boonsboro, MD) https://t.co/VgIUWkgBNX https://t.co/lJID0c8JfE'},
 {'user_id': 22088130,
  'user_name': 'yoursunny',
  'tweet_id': '1655687697862729728',
  'full_text': "Closed! (@ Litton's Produce and Berries in Fairplay, MD) https://t.co/3fCqPio3jR"},
 {'user_id': 22088130,
  'user_name': 'yoursunny',
  'tweet_id': '1655677298782109702',
  'full_text': "🦀

In [ ]:
from google.colab import files
files.download('/content/tweets.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open("tweets.json", "w") as f:
    json.dump(all_tweets, f)

In [ ]:
with open("all_tweets.json", "w") as f:
    json.dump(all_tweets, f)

files.download('/content/all_tweets.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Tweet Classification
#playing around with models - not the pipeline

In [ ]:
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.8 MB/s eta 0:00:00


In [ ]:
model_name = 'textattack/bert-base-uncased-AG-news'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
# Classify text practise
text = "new iphone has been launched"
encoded_text = tokenizer.encode_plus(
    text,
    max_length=512,
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)
with torch.no_grad():
    output = model(**encoded_text)

predicted_class = torch.argmax(output.logits)
labels = ['World', 'Sports', 'Business', 'Science/Technology']
print(labels[predicted_class])

Science/Technology


# Sentiment Analysis
#playing around with models - not the pipeline

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
!pip install nltk
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# Analyze the sentiment of a sentence

entence = "The movie was really good!"
scores = analyzer.polarity_scores(data[0]['full_text'])

# Print the sentiment scores
print(scores)

{'neg': 0.0, 'neu': 0.936, 'pos': 0.064, 'compound': 0.4576}


# Pipeline
#Categorizing



In [ ]:
with open('/content/all_tweets.json', 'r') as f:
    # Load the JSON data
    data = json.load(f)

In [ ]:
data[0].keys()

dict_keys(['user_id', 'user_name', 'tweet_id', 'full_text', 'replies'])

In [ ]:
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = 'textattack/bert-base-uncased-AG-news'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
categories = []
#analysis = {"categories": [], "sentiment": []}
labels = ['World', 'Sports', 'Business', 'Science/Technology']
for tweet in data:
  tweet_text = tweet['full_text']
  encoded_text = tokenizer.encode_plus(
    tweet_text,
    max_length=512,
    truncation=True,
    padding='max_length',
    return_tensors='pt')
  with torch.no_grad():
    output = model(**encoded_text)
  predicted_class = torch.argmax(output.logits)
  categories.append(labels[predicted_class])

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Create a SparkSession object
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TweetClassification").getOrCreate()

# Load the BERT tokenizer and model
model_name = 'textattack/bert-base-uncased-AG-news'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Define a function to classify a tweet
def classify_tweet(tweet_text):
    encoded_text = tokenizer.encode_plus(
        tweet_text,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_tensors='pt')
    with torch.no_grad():
        output = model(**encoded_text)
    predicted_class = torch.argmax(output.logits)
    categories = ['World', 'Sports', 'Business', 'Science/Technology']
    return categories[predicted_class]

classify_tweet_udf = udf(classify_tweet, StringType())

# Read the JSON file into a Spark DataFrame
json_file_path = "/content/all_tweets.json"
df = spark.read.json(json_file_path)

# Apply the UDF to classify the tweets
df = df.withColumn("category", classify_tweet_udf(df.full_text))
df =
# Show the resulting DataFrame
category = [row.category for row in df.collect()]

In [ ]:
category

In [ ]:
full_text = []
reps = []

for tweet in data:
  tweet_text = tweet['full_text']
  full_text.append(tweet_text)
  reps.append(tweet['replies'])

In [ ]:
import pandas as pd
df = pd.DataFrame({'category': categories['Category'], 'tweet': full_text,'column3': reps})
df.to_excel("category-tweets-replies.xlsx", index=False)

In [ ]:
## dont run
tweets = []
for tweet in data:
  tweet_text = tweet['full_text']
  tweets.append(tweet_text)

df = pd.DataFrame(tweets, columns=["tweet"])
df.to_excel("latest tweets.xlsx", index=False)

In [ ]:
## dont run
import pandas as pd
df = pd.DataFrame(categories, columns=["Category"])
df.to_excel("latest categories.xlsx", index=False)

In [ ]:
## dont run
categories = pd.read_excel('/content/latest categories.xlsx', sheet_name='Sheet1')

In [ ]:
## dont run
categories['Category']

In [ ]:
print(len(categories),len(full_text),len(reps))

180 180 180


59

In [ ]:
column_series = df['column2']
column_series.to_csv('tweets.txt', index=False, sep='\n')

In [ ]:
column_series = df['column1']
column_series.to_csv('categories.txt', index=False, sep='\n')

In [ ]:
df.to_excel('output.xlsx', index=False)

In [ ]:
df = pd.read_excel('/content/output.xlsx')
ids = df['column2'].tolist()
len(ids)

59

# Pipeline
# Toxic Tweet Detection

In [ ]:
tweets_reps_categories = pd.read_excel('/content/category-tweets-replies.xlsx')

In [ ]:
tweets_reps_categories.loc[len(tweets_reps_categories)] = ["","a toxic tweet - i hate you so much go die",""]

In [ ]:
!pip install detoxify
from detoxify import Detoxify

In [ ]:
toxic_tweets = []
for index, tweet in tweets_reps_categories.iterrows():
    preds = Detoxify('original').predict(tweet['tweet'])
    sorted_dict = sorted(preds.items(), key=lambda x: x[1], reverse=True)
    highest_two = sorted_dict[:2]
    toxicity = highest_two[0][1] + highest_two[1][1]
    if toxicity>=0.8:
      toxic_tweets.append(tweet['tweet'])
      tweets_reps_categories.drop(index, inplace=True)

In [ ]:
from pyspark import SparkContext, SparkConf
from detoxify import Detoxify
import csv

conf = SparkConf().setAppName("SentimentAnalysis")
sc = SparkContext.getOrCreate(conf)

# Initialize Detoxify model
model = Detoxify('original')

# Define the function to apply to each tweet
def tell_sentiment(tweet):
    preds = model.predict(tweet)
    return sorted(preds.items(), key=lambda x: x[1], reverse=True)

tweets = tweets_reps_categories['tweet']
tweets_rdd = sc.parallelize(tweets)
# tweets_rdd = sc.textFile("/content/followings.csv").map(lambda line: line.split("\t"))
# bios_rdd = tweets_rdd.map(lambda fields: fields[5])

sentiments_rdd = tweets_rdd.map(tell_sentiment)

# Collect the results to the driver program and print them
sentiment_returns = sentiments_rdd.collect()

# Stop the Spark context
sc.stop()

In [ ]:
toxic_tweets = []
i = 0
for index, tweet in tweets_reps_categories.iterrows():
    highest_two = sentiment_returns[i][:2]
    i+=1
    toxicity = highest_two[0][1] + highest_two[1][1]
    if toxicity>=0.8:
      toxic_tweets.append(tweet['tweet'])
      tweets_reps_categories.drop(index, inplace=True)

In [ ]:
def tell_sentiment(tweet):
  preds = Detoxify('original').predict(tweet)
  return sorted(preds.items(), key=lambda x: x[1], reverse=True)

In [ ]:
toxic_tweet = "a toxic tweet - i hate you so much go die"
ret = tell_sentiment(toxic_tweet)

# Pipeline
#Replies Analysis


In [ ]:
# Analyze the sentiment of a sentence
#!pip install nltk
import pandas as pd
import numpy as np
#from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
analyzer = SentimentIntensityAnalyzer()
sentence = "I went to swimming yesterday and it was amazing! it was a good exercise. Good for health"
scores = analyzer.polarity_scores(sentence)

# Print the sentiment scores
print(scores)

{'neg': 0.0, 'neu': 0.548, 'pos': 0.452, 'compound': 0.8718}


In [ ]:
for index, tweet in tweets_reps_categories.iterrows():
  replies = tweet['column3']
  neg = 0
  pos = 0
  for reply in replies:
    scores = analyzer.polarity_scores(reply)
    neg+=scores['neg']
    pos+=scores['compound']
  if pos<1.5*neg:
    toxic_tweets.append(tweet['tweet'])
    tweets_reps_categories.drop(index, inplace=True)

In [ ]:
tweets_reps_categories

save in seperate files

In [ ]:
tweets_reps_categories.head(5)

,category,tweet,column3
0,World,Thoughts on Para 28 (58:01-66:12):\nWhile the ...,"['unroll', ""because it is named after a woman ..."
1,Science/Technology,At a massive EdTech conference &amp; the tone ...,['I think the singularity happened when target...
2,Science/Technology,The #FABRICtestbed team understands that not e...,[]
3,Science/Technology,"RT @thebsdbox: Pretty tired, but also super pu...",[]
4,Science/Technology,How much time does twitter blue take in review...,"['Upto 7 days!', 'For me they took 7 working d..."


In [ ]:
print(tweets_reps_categories['category'].unique())

['World' 'Science/Technology' 'Science/World' 'Sports' 'Business'
 'Buisness']


In [ ]:
#['World', 'Sports', 'Business', 'Science/Technology']
buisness = []
science = []
world = []
sports = []

for index, tweet in tweets_reps_categories.iterrows():
  if tweet['category']=="Business":
    buisness.append(tweet['tweet'])
  elif tweet['category']=="Sports":
    sports.append(tweet['tweet'])
  elif tweet['category']=="Science/Technology":
    science.append(tweet['tweet'])
  elif tweet['category']=="world":
    world.append(tweet['tweet'])



# ChatGPT

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.0 MB/s eta 0:00:00


In [ ]:
!touch "/content/OpenAI.json"

In [ ]:
import json

OPENAI_API_KEY = ''
with open('/content/OpenAI.json', 'r') as file_to_read:
    json_data = json.load(file_to_read)
    OPENAI_API_KEY = json_data["OPENAI_API_KEY"]

In [ ]:
import os
import openai

openai.api_key =  OPENAI_API_KEY

In [ ]:
sports

"👈\n\n1. SF's Mayor London Breed introduces wide-spanning housing reforms\n2. MartelMetellus offers a system for training efficiently in under 3 minutes\n3. 1437_MT8 is the Season 3 champion\n4. KuCoin is holding a quiz with the chance to win community bonuses"

In [ ]:
buisness = []
science = []
world = []
sports = []

In [ ]:
prompt = "Please summarize the following lines :\n" + "\n".join(science)
response = openai.Completion.create(
      engine="text-davinci-002",
      prompt=prompt,
      max_tokens=100,
      n=1,
      stop=None,
      temperature=0.5,
  )

science = response.choices[0].text.strip()
print(science)

In [ ]:
world = ["The Para 28 of Surah 'Maryam' contains two Surah titles referring to women, PTI has issued tickets for all 297 seats of the Punjab Assembly",
"A blog post suggests top places to explore in Naran Kaghan Valley in 2023",
"A tweet encourages people to not be afraid to change for the better and not mock others who are trying to do the same.",
"A thread talks about the evolution of music listening devices over the years.",
"A tweet mentions the arrest of PTI Instagram Lead Atta Ur Rehman.",
"Easypaisa has faced losses for the fifth consecutive year, despite having a massive footprint in digital payments.",
"There are various retweets of other people's tweets that contain a mix of news, politics, and technology-related content."]

In [ ]:
science = ["Education leaders at a conference discussed how AI has made the future of jobs and education unknowable\n",
"The #FABRICtestbed team is offering a KNIT 6 STREAMING PASS for specific events for those who can't attend in-person\n",
"There is an underrated comment about sqlite3_reset().",
"The winning projects of Napptive's hackathon have been posted on their blog.",
"Your iPhone and passcode can lock you out of your Apple account and content.",
"There is a tutorial of 170+ slides on supercharging academic writing with AI-powered apps.",
"TOI is generating stories with AI.",
"Someone is now friends with @phindsearch and not @ChatGPT.",
"There is a Hover Button tutorial using HTML and CSS."]

In [ ]:
final_prompts = {}
final_prompts['buisness']=sum_buisness

In [ ]:
final_prompts['sports']=sum_sports

In [ ]:
final_prompts['world'] = world

In [ ]:
final_prompts['science'] = science

In [ ]:
json_str = json.dumps(final_prompts)

# Write JSON string to a file
with open('prompts.json', 'w') as f:
    f.write(json_str)

In [ ]:
import json

# define a dictionary

# open a file in write mode
with open("prompts.json", "w") as file:
    # write the dictionary to the file in JSON format
    json.dump(final_prompts, file)

In [ ]:
def summarize_tweets(tweets):
    # Join the tweets into a single string
    text = " ".join(tweets)

    # Use the OpenAI API to summarize the text
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=f"Please summarize the following tweets:\n\n{text}",
        max_tokens=512,
        n=1,
        stop=None,
        temperature=0.5,
    )
    summary = response.choices[0].text.strip()

    # Split the summary into bullet points
    points = []
    for sentence in summary.split(". "):
        if sentence not in points:
            points.append(sentence)

    # Print the bullet points
    print("Main points:")
    for point in points:
        print("- " + point)

# Example tweets to summarize
tweets = [
    "Thoughts on Para 28 (58:01-66:12): While the Surah, 'Maryam' (Mary), is the only Surah with a female name, two Surah titles in this Para refer to women too:",
    "At a massive EdTech conference &amp; the tone among many education leaders is that The Singularity happened (without the need for super intelligence)  AI has made the future unknowable: what jobs will we train folks for? How will education work? We will figure it out, but for now 🤷‍♂️",
]

# Summarize the tweets
summarize_tweets(tweets)

Main points:
- 1) At a massive EdTech conference, the tone among many education leaders is that The Singularity happened (without the need for super intelligence)
2) AI has made the future unknowable: what jobs will we train folks for? How will education work?
3) We will figure it out, but for now, people are unsure about what the future holds.


# Anvil WebAPP

In [ ]:
!pip install anvil-uplink
import anvil.server

anvil.server.connect("server_####")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=b82587a93b19a2286430213fe9a94ffe9ddd306e076677ccf93ede2bbde9e18a
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
@anvil.server.callable
def send_prompts():
  return final_prompts

In [ ]:
final_prompts.keys()

dict_keys(['buisness', 'sports', 'world', 'science'])

In [ ]:
@anvil.server.callable
def send_filtered_out_tweets():
  return toxic_tweet[0]

In [ ]:
toxic_tweets[0]